In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df_u = pd.read_csv('takehome_users.csv', encoding='latin-1', index_col=0)
df_u

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
object_id,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
...,...,...,...,...,...,...,...,...,...
11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,1.378448e+09,0,0,89,8263.0
11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,NaN
11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,1.398603e+09,1,1,83,8074.0


In [27]:
df_u.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   creation_time               12000 non-null  object 
 1   name                        12000 non-null  object 
 2   email                       12000 non-null  object 
 3   creation_source             12000 non-null  object 
 4   last_session_creation_time  8823 non-null   float64
 5   opted_in_to_mailing_list    12000 non-null  int64  
 6   enabled_for_marketing_drip  12000 non-null  int64  
 7   org_id                      12000 non-null  int64  
 8   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(3), object(4)
memory usage: 937.5+ KB


In [3]:
df_eng = pd.read_csv('/content/takehome_user_engagement.csv')
df_eng

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
...,...,...,...
207912,2013-09-06 06:14:15,11996,1
207913,2013-01-15 18:28:37,11997,1
207914,2014-04-27 12:45:16,11998,1
207915,2012-06-02 11:55:59,11999,1


In [4]:
df_eng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [5]:
# convert from string to datetime format
df_eng['time_stamp'] = pd.to_datetime(df_eng['time_stamp'])
df_eng = df_eng.set_index('time_stamp')
df_eng

,user_id,visited
time_stamp,,
2014-04-22 03:53:30,1,1
2013-11-15 03:45:04,2,1
2013-11-29 03:45:04,2,1
2013-12-09 03:45:04,2,1
2013-12-25 03:45:04,2,1
...,...,...
2013-09-06 06:14:15,11996,1
2013-01-15 18:28:37,11997,1
2014-04-27 12:45:16,11998,1


Now that we have a datetime index for the dataframe, I can use the .resample() method to convert the dates into weeks. While this won't perfectly capture every 7-day period in which a user may have adopted the product, it should capture the lion's share of the 3-times-a-week users and therefore should suffice for the purposes of this cursory analysis. I will group the data by 'user_id' first, then resample the dates, and sum the data so that the 'visited' column will show us which users are 'adopted':


In [7]:
df_eng = df_eng.groupby('user_id').resample('1W').sum()
df_eng.head(10)

user_id  visited
user_id time_stamp                  
1       2014-04-27        1        1
2       2013-11-17        2        1
        2013-11-24        0        0
        2013-12-01        2        1
        2013-12-08        0        0
        2013-12-15        2        1
        2013-12-22        0        0
        2013-12-29        2        1
        2014-01-05        2        1
        2014-01-12        2        1

In [9]:
# create dataframe of only the adopted users
df_adopted = df_eng[df_eng['visited'] > 2]
del df_adopted['user_id']
df_adopted

visited
user_id time_stamp         
2       2014-02-09        3
10      2013-03-03        3
        2013-04-14        3
        2013-04-28        3
        2013-05-05        4
...                     ...
11988   2014-04-06        3
        2014-05-04        4
        2014-05-18        3
        2014-05-25        5
        2014-06-01        3

[33859 rows x 1 columns]

In [14]:
# creating user id list for adopted users
adopted_index = list(df_adopted.index)
adopted_users_list = []
for i in range(len(adopted_index)):
    adopted_users_list.append(adopted_index[i][0])
len(adopted_users_list)

33859

In [15]:
adopted_users = set(adopted_users_list)
len(adopted_users)

1445

In [16]:
# adding an adopted column to the dataframe with a default value of False
df_u['adopted'] = False

# use the adopted_users set to label the adopted users in the dataframe
for user in adopted_users:
    df_u.loc[user, 'adopted'] = True

df_u.head(10)

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted
object_id,,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,False
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,True
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,False
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,False
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,False
6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1.387424e+09,0,0,197,11241.0,False
7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN,False
8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN,False
9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN,False


In [37]:
df_u.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   creation_time               12000 non-null  object 
 1   name                        12000 non-null  object 
 2   email                       12000 non-null  object 
 3   creation_source             12000 non-null  object 
 4   last_session_creation_time  8823 non-null   float64
 5   opted_in_to_mailing_list    12000 non-null  int64  
 6   enabled_for_marketing_drip  12000 non-null  int64  
 7   org_id                      12000 non-null  int64  
 8   invited_by_user_id          6417 non-null   float64
 9   adopted                     12000 non-null  bool   
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 1.2+ MB


In [38]:
df_u['invited_by_user_id'].value_counts(ascending=False, dropna=False)

NaN        5583
10741.0      13
2527.0       12
1525.0       11
2308.0       11
           ... 
2071.0        1
1390.0        1
5445.0        1
8526.0        1
5450.0        1
Name: invited_by_user_id, Length: 2565, dtype: int64

In [17]:
df_u['invited_by_user'] = df_u['invited_by_user_id'].isnull() == False
del df_u['invited_by_user_id']
df_u

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted,invited_by_user
object_id,,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,False,True
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,True,True
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,False,True
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,False,True
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,False,True
...,...,...,...,...,...,...,...,...,...,...
11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,1.378448e+09,0,0,89,False,True
11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,False,False
11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,1.398603e+09,1,1,83,False,True


In [18]:
# split the individual email address from email domain
df_u['email_split'] = df_u['email'].str.split('@')
del df_u['email']

# extract the email domains into separate column
for user in range(len(df_u)):
    df_u.loc[user+1, 'email_domain'] = df_u.loc[user+1, 'email_split'][1]
del df_u['email_split']

# evaluate the most common email domains
df_u['email_domain'].value_counts()[:10]

gmail.com         3562
yahoo.com         2447
jourrapide.com    1259
cuvox.de          1202
gustr.com         1179
hotmail.com       1165
rerwl.com            2
oqpze.com            2
qgjbc.com            2
dqwln.com            2
Name: email_domain, dtype: int64

In [19]:
# create final email column
df_u['email'] = 'other'
df_u

,creation_time,name,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted,invited_by_user,email_domain,email
object_id,,,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,GUEST_INVITE,1.398139e+09,1,0,11,False,True,yahoo.com,other
2,2013-11-15 03:45:04,Poole Matthew,ORG_INVITE,1.396238e+09,0,0,1,True,True,gustr.com,other
3,2013-03-19 23:14:52,Bottrill Mitchell,ORG_INVITE,1.363735e+09,0,0,94,False,True,gustr.com,other
4,2013-05-21 08:09:28,Clausen Nicklas,GUEST_INVITE,1.369210e+09,0,0,1,False,True,yahoo.com,other
5,2013-01-17 10:14:20,Raw Grace,GUEST_INVITE,1.358850e+09,0,0,193,False,True,yahoo.com,other
...,...,...,...,...,...,...,...,...,...,...,...
11996,2013-09-06 06:14:15,Meier Sophia,ORG_INVITE,1.378448e+09,0,0,89,False,True,gustr.com,other
11997,2013-01-10 18:28:37,Fisher Amelie,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,False,False,gmail.com,other
11998,2014-04-27 12:45:16,Haynes Jake,GUEST_INVITE,1.398603e+09,1,1,83,False,True,cuvox.de,other


In [20]:
# populate the final email column with the common email domains
for user in range(len(df_u)):
    if df_u.loc[user+1, 'email_domain'] == 'gmail.com':
        df_u.loc[user+1, 'email'] = 'gmail.com'
    if df_u.loc[user+1, 'email_domain'] == 'yahoo.com':
        df_u.loc[user+1, 'email'] = 'yahoo.com'
    if df_u.loc[user+1, 'email_domain'] == 'jourrapide.com':
        df_u.loc[user+1, 'email'] = 'jourrapide.com'
    if df_u.loc[user+1, 'email_domain'] == 'cuvox.de':
        df_u.loc[user+1, 'email'] = 'cuvox.de'
    if df_u.loc[user+1, 'email_domain'] == 'gustr.com':
        df_u.loc[user+1, 'email'] = 'gustr.com'
    if df_u.loc[user+1, 'email_domain'] == 'hotmail.com':
        df_u.loc[user+1, 'email'] = 'hotmail.com'

del df_u['email_domain']

df_u['email'].value_counts(ascending=False)

gmail.com         3562
yahoo.com         2447
jourrapide.com    1259
cuvox.de          1202
other             1186
gustr.com         1179
hotmail.com       1165
Name: email, dtype: int64

In [21]:
df_ml = pd.DataFrame({'adopted_user': df_u['adopted'].values}, index=df_u.index)
df_ml

,adopted_user
object_id,
1,False
2,True
3,False
4,False
5,False
...,...
11996,False
11997,False
11998,False


In [22]:
df_ml['mailing_list'] = df_u['opted_in_to_mailing_list']
df_ml['marketing_drip'] = df_u['enabled_for_marketing_drip']
df_ml['invited_by_user'] = df_u['invited_by_user']
df_ml

,adopted_user,mailing_list,marketing_drip,invited_by_user
object_id,,,,
1,False,1,0,True
2,True,0,0,True
3,False,0,0,True
4,False,0,0,True
5,False,0,0,True
...,...,...,...,...
11996,False,0,0,True
11997,False,0,0,False
11998,False,1,1,True


In [23]:
# convert org_id column from int to string
df_u['org_id'] = df_u['org_id'].astype('str')

# create the dummy variables
df_dummies = pd.get_dummies(df_u[['creation_source', 'org_id', 'email']])

# add dummy variables to df_model dataframe
df_ml = pd.concat([df_ml, df_dummies], axis=1)
df_ml.head()

,adopted_user,mailing_list,marketing_drip,invited_by_user,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH,org_id_0,org_id_1,org_id_10,org_id_100,org_id_101,org_id_102,org_id_103,org_id_104,org_id_105,org_id_106,org_id_107,org_id_108,org_id_109,org_id_11,org_id_110,org_id_111,org_id_112,org_id_113,org_id_114,org_id_115,org_id_116,org_id_117,org_id_118,org_id_119,org_id_12,org_id_120,org_id_121,org_id_122,org_id_123,org_id_124,org_id_125,...,org_id_7,org_id_70,org_id_71,org_id_72,org_id_73,org_id_74,org_id_75,org_id_76,org_id_77,org_id_78,org_id_79,org_id_8,org_id_80,org_id_81,org_id_82,org_id_83,org_id_84,org_id_85,org_id_86,org_id_87,org_id_88,org_id_89,org_id_9,org_id_90,org_id_91,org_id_92,org_id_93,org_id_94,org_id_95,org_id_96,org_id_97,org_id_98,org_id_99,email_cuvox.de,email_gmail.com,email_gustr.com,email_hotmail.com,email_jourrapide.com,email_other,email_yahoo.com
object_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,False,1,0,True,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,True,0,0,True,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,False,0,0,True,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
4,False,0,0,True,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,False,0,0,True,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [24]:
# divide the data into label and features for use in ml models
X = df_ml.iloc[:, 1:]
y = df_ml.loc[:, 'adopted_user']

# split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6, stratify=y)

In [25]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(class_weight='balanced_subsample', random_state=6)
rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_test)
print('Accuracy on training set = {}'.format(rfc.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(rfc.score(X_test, y_test)))

Accuracy on training set = 0.9426666666666667
Accuracy on test set = 0.8366666666666667


In [26]:
# generate classification report of model performance
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_rfc))

              precision    recall  f1-score   support

       False       0.88      0.94      0.91      2639
        True       0.18      0.10      0.13       361

    accuracy                           0.84      3000
   macro avg       0.53      0.52      0.52      3000
weighted avg       0.80      0.84      0.82      3000



In [27]:
# generate confusion matrix for model performance
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_rfc))

[[2474  165]
 [ 325   36]]


In [28]:
# create dataframe of feature importances from model
df_features = pd.DataFrame({'rfc': rfc.feature_importances_}, index=df_ml.columns[1:])

# sort by highest values
df_features.sort_values('rfc', ascending=False)

,rfc
mailing_list,0.036456
marketing_drip,0.024465
email_gmail.com,0.016984
email_gustr.com,0.013560
email_yahoo.com,0.013497
...,...
org_id_272,0.000114
org_id_389,0.000109
org_id_412,0.000101
org_id_355,0.000055


While at first blush it may appear that the "out-of-the-box" Random Forest Classifier performed well by classifying 84% of the test set samples correctly, we must keep in mind that the null accuracy rate is 88% (if the model simply predicted that every user did not adopt the product). When data is imbalanced to the degree that we have here, it is more important to analyze how well the model was able to identify the minority class (in this case, the adopted users) as models tend to favor majority classes when data is imbalanced. Focusing on identifying the minority class of adopted users is particularly important here given that the goal of this project is to identify which of the provided features is most predictive of future user product adoption.

Here, the classification report and confusion matrix demonstrates that the "out-of-the-box" model correctly identified adopted users at an alarmingly low rate (only 29 of 361 in the test set). The feature importance values of the model therefore are not going to provide much insight as to which of the features are most predictive of future user product adoption.

All is not lost, however. The Random Forest Classifier has a number of hyperparameters that can be tuned to improve performance. Let's tune some of them to see if it improves the model's accuracy rate on the adopted users:

In [29]:
# tune some of the hyperparameters with gridsearch
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier(random_state=6)
param_grid = {'n_estimators': [100, 300, 500], 'max_depth': [5, 15, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 3, 5]}
grid = GridSearchCV(rfc, param_grid, cv=3, scoring='accuracy')
grid.fit(X_train, y_train)
y_pred_rfc = grid.predict(X_test)
print(grid.score(X_test, y_test))

0.8796666666666667


In [30]:
grid.best_score_

0.8795555555555555

In [31]:
grid.best_estimator_

RandomForestClassifier(max_depth=5, random_state=6)

In [32]:
# train the model and make predictions on the training and test sets
from sklearn.ensemble import RandomForestClassifier
rfc_tuned = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=2, min_samples_leaf=1, criterion='gini', class_weight='balanced_subsample', random_state=6)
rfc_tuned.fit(X_train, y_train)
y_pred_rfc_tuned = rfc_tuned.predict(X_test)
print('Accuracy on training set = {}'.format(rfc_tuned.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(rfc_tuned.score(X_test, y_test)))

Accuracy on training set = 0.6206666666666667
Accuracy on test set = 0.6


In [33]:
# generate classification report of model performance
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_rfc_tuned))

              precision    recall  f1-score   support

       False       0.90      0.61      0.73      2639
        True       0.15      0.51      0.23       361

    accuracy                           0.60      3000
   macro avg       0.53      0.56      0.48      3000
weighted avg       0.81      0.60      0.67      3000



In [34]:
# generate confusion matrix for model performance
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_rfc_tuned))

[[1616 1023]
 [ 177  184]]


While the overall accuracy of the tuned classifier decreased significantly, the confusion matrix demonstrates that the model performed significantly better in identifying over half of the adopted users accurately (183 of 361 in the test set). We therefore should be able to gain insight as to which particular features are most predictive of future user adoption by seeing which of the features received the highest feature importance increase by the tuned model. Let's take a look:

In [35]:
# add the tuned classifier feature importance values to the df_features dataframe
df_features['rfc_tuned'] = rfc_tuned.feature_importances_

# add column calculating the difference between the feature importance values
df_features['difference'] = df_features['rfc_tuned'] - df_features['rfc']

# sort by highest difference
df_features.sort_values('difference', ascending=False)[:15]

,rfc,rfc_tuned,difference
creation_source_PERSONAL_PROJECTS,0.011761,0.080421,0.068660
org_id_0,0.008950,0.065023,0.056073
creation_source_GUEST_INVITE,0.008247,0.063778,0.055531
email_yahoo.com,0.013497,0.044497,0.031000
email_hotmail.com,0.010490,0.037771,0.027281
creation_source_SIGNUP_GOOGLE_AUTH,0.005618,0.022810,0.017192
email_gmail.com,0.016984,0.033124,0.016140
org_id_1,0.007135,0.022089,0.014954
org_id_3,0.006261,0.017236,0.010975
creation_source_ORG_INVITE,0.009566,0.019857,0.010291


**Final Analysis/Conclusions**

The above table demonstrates that the factors that most likely lead to a particular user adopting the product are when the user is invited to join another user's workspace or is invited to an organization as a guest. It also demonstrates that being a member of certain organizations, and in particular 'org_id_0,' makes it more likely for a user to adopt the product. The tuned classifier also found three of the email domains to be of increased importance (and signing up using google authentication), as well as being invited to become a full member of an organization.

Based on the above, I would make the following recommendations to Relax regarding their attempts to improve future user adoption:









 

*   Run promotions seeking to incentivize users to invite prospective users to join their personal workspaces on the product;
*   Run promotions seeking to incentivize organizations to grow by sending guest and/or full invites to prospective users to join the product;
*   Promote certain organizations identified by the model above (in particular org_id_0) to all users; and
* Target its regular email marketing drip to existing and prospective users with yahoo.com, hotmail.com, and gmail.com email addresses.





